In [1]:
import xarray as xr
from dask.distributed import Client
import numpy as np
import snuggs
from rio_color.operations import parse_operations
from rio_color.colorspace import ColorSpace
import pystac

In [2]:
item = pystac.read_file("https://stac.core.eopf.eodc.eu/collections/sentinel-2-l1c/items/S2B_MSIL1C_20250113T103309_N0511_R108_T32TLQ_20250113T122458")

In [3]:
item.assets

{'SR_10m': <Asset href=https://objectstore.eodc.eu:2222/e05ab01a9d56408d82ac32d69a5aae2a:sample-data/tutorial_data/cpm_v253/S2B_MSIL1C_20250113T103309_N0511_R108_T32TLQ_20250113T122458.zarr/measurements/reflectance/r10m>,
 'SR_20m': <Asset href=https://objectstore.eodc.eu:2222/e05ab01a9d56408d82ac32d69a5aae2a:sample-data/tutorial_data/cpm_v253/S2B_MSIL1C_20250113T103309_N0511_R108_T32TLQ_20250113T122458.zarr/measurements/reflectance/r20m>,
 'SR_60m': <Asset href=https://objectstore.eodc.eu:2222/e05ab01a9d56408d82ac32d69a5aae2a:sample-data/tutorial_data/cpm_v253/S2B_MSIL1C_20250113T103309_N0511_R108_T32TLQ_20250113T122458.zarr/measurements/reflectance/r60m>,
 'B01_60m': <Asset href=https://objectstore.eodc.eu:2222/e05ab01a9d56408d82ac32d69a5aae2a:sample-data/tutorial_data/cpm_v253/S2B_MSIL1C_20250113T103309_N0511_R108_T32TLQ_20250113T122458.zarr/measurements/reflectance/r60m/b01>,
 'B02_10m': <Asset href=https://objectstore.eodc.eu:2222/e05ab01a9d56408d82ac32d69a5aae2a:sample-data/tutor

In [ ]:
remote_product_path = item.get_assets().get("product").href 

dt = xr.open_datatree(remote_product_path, engine="zarr", chunks={})

/tmp/ipykernel_2740130/1252326399.py:2: FutureWarning: In a future version of xarray decode_timedelta will default to False rather than None. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  dt = xr.open_datatree(remote_product_path, engine="zarr", chunks={})
/tmp/ipykernel_2740130/1252326399.py:2: FutureWarning: In a future version of xarray decode_timedelta will default to False rather than None. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  dt = xr.open_datatree(remote_product_path, engine="zarr", chunks={})


In [5]:
client = Client()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 16,Total memory: 30.15 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:38683,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 16
Started: Just now,Total memory: 30.15 GiB
Comm: tcp://127.0.0.1:45023,Total threads: 4
Dashboard: http://127.0.0.1:46633/status,Memory: 7.54 GiB
Nanny: tcp://127.0.0.1:33597,


In [6]:


# remote_product_path = "https://objectstore.eodc.eu:2222/e05ab01a9d56408d82ac32d69a5aae2a:sample-data/tutorial_data/cpm_v253/S2B_MSIL1C_20250113T103309_N0511_R108_T32TLQ_20250113T122458.zarr"
# dt = xr.open_datatree(remote_product_path, engine="zarr", chunks={})

In [7]:
b01 = dt["measurements/reflectance/r60m"]["b01"].chunk({"x": 512, "y": 512})

b04 = dt["measurements/reflectance/r10m"]["b04"].chunk({"x": 512, "y": 512})

red = dt["measurements/reflectance/r10m"]["b04"].chunk({"x": 512, "y": 512})
green = dt["measurements/reflectance/r10m"]["b03"].chunk({"x": 512, "y": 512})
blue = dt["measurements/reflectance/r10m"]["b02"].chunk({"x": 512, "y": 512})
nir = dt["measurements/reflectance/r10m"]["b08"].chunk({"x": 512, "y": 512})
swir16 = dt["measurements/reflectance/r20m"]["b11"].chunk({"x": 512, "y": 512})

In [8]:
swir16

<xarray.DataArray 'b11' (y: 5490, x: 5490)> Size: 241MB
dask.array<rechunk-merge, shape=(5490, 5490), dtype=float64, chunksize=(512, 512), chunktype=numpy.ndarray>
Coordinates:
  * x        (x) int64 44kB 300010 300030 300050 300070 ... 409750 409770 409790
  * y        (y) int64 44kB 5000030 5000010 4999990 ... 4890290 4890270 4890250
Attributes:
    _eopf_attrs:     {'add_offset': -0.1, 'coordinates': ['x', 'y'], 'dimensi...
    dtype:           <u2
    fill_value:      0
    long_name:       TOA reflectance from MSI acquisition at spectral band b1...
    proj:bbox:       [300000.0, 4890240.0, 409800.0, 5000040.0]
    proj:epsg:       32632
    proj:shape:      [5490, 5490]
    proj:transform:  [20.0, 0.0, 300000.0, 0.0, -20.0, 5000040.0, 0.0, 0.0, 1.0]
    proj:wkt2:       PROJCS["WGS 84 / UTM zone 32N",GEOGCS["WGS 84",DATUM["WG...
    units:           digital_counts
    valid_max:       65535
    valid_min:       1

In [9]:
dt["measurements/reflectance/r10m"].keys()

KeysView(<xarray.DataTree 'r10m'>
Group: /measurements/reflectance/r10m
    Dimensions:  (y: 10980, x: 10980)
    Coordinates:
      * x        (x) int64 88kB 300005 300015 300025 300035 ... 409775 409785 409795
      * y        (y) int64 88kB 5000035 5000025 5000015 ... 4890265 4890255 4890245
    Data variables:
        b02      (y, x) float64 964MB dask.array<chunksize=(1830, 1830), meta=np.ndarray>
        b03      (y, x) float64 964MB dask.array<chunksize=(1830, 1830), meta=np.ndarray>
        b04      (y, x) float64 964MB dask.array<chunksize=(1830, 1830), meta=np.ndarray>
        b08      (y, x) float64 964MB dask.array<chunksize=(1830, 1830), meta=np.ndarray>)

In [10]:
dt["measurements/reflectance/r60m"]["b01"].shape

(1830, 1830)

In [11]:
dt["measurements/reflectance/r10m"]["b04"].shape

(10980, 10980)

In [12]:
dt["measurements/reflectance/r10m"]["b04"].dims


('y', 'x')

In [13]:
dt["measurements/reflectance/r10m"]["b04"].coords

Coordinates:
  * x        (x) int64 88kB 300005 300015 300025 300035 ... 409775 409785 409795
  * y        (y) int64 88kB 5000035 5000025 5000015 ... 4890265 4890255 4890245

In [14]:
dt["measurements/reflectance/r10m"]["b04"].coords

Coordinates:
  * x        (x) int64 88kB 300005 300015 300025 300035 ... 409775 409785 409795
  * y        (y) int64 88kB 5000035 5000025 5000015 ... 4890265 4890255 4890245

In [15]:
swir16_resampled = swir16.interp(x=red["x"], y=b04["y"], method="nearest").chunk({"x": 512, "y": 512})

In [16]:
swir16_resampled

<xarray.DataArray 'b11' (y: 10980, x: 10980)> Size: 964MB
dask.array<rechunk-merge, shape=(10980, 10980), dtype=float64, chunksize=(512, 512), chunktype=numpy.ndarray>
Coordinates:
  * x        (x) int64 88kB 300005 300015 300025 300035 ... 409775 409785 409795
  * y        (y) int64 88kB 5000035 5000025 5000015 ... 4890265 4890255 4890245
Attributes:
    _eopf_attrs:     {'add_offset': -0.1, 'coordinates': ['x', 'y'], 'dimensi...
    dtype:           <u2
    fill_value:      0
    long_name:       TOA reflectance from MSI acquisition at spectral band b1...
    proj:bbox:       [300000.0, 4890240.0, 409800.0, 5000040.0]
    proj:epsg:       32632
    proj:shape:      [5490, 5490]
    proj:transform:  [20.0, 0.0, 300000.0, 0.0, -20.0, 5000040.0, 0.0, 0.0, 1.0]
    proj:wkt2:       PROJCS["WGS 84 / UTM zone 32N",GEOGCS["WGS 84",DATUM["WG...
    units:           digital_counts
    valid_max:       65535
    valid_min:       1

In [17]:
#b01_resampled.plot()

In [18]:
def normalized_difference(band1, band2):
    return (band1 - band2) / (band1 + band2)


In [19]:
ndvi = xr.apply_ufunc(
        normalized_difference,
        red,
        nir,
        dask="parallelized",
        output_dtypes=[np.float32],
    )


In [20]:
ndwi = xr.apply_ufunc(
        normalized_difference,
        green,
        nir,
        dask="parallelized",
        output_dtypes=[np.float32],
    )

In [21]:
red.values

array([[1.0102, 0.9854, 0.984 , ..., 0.0726, 0.0855, 0.0851],
       [1.0071, 0.9648, 0.9357, ..., 0.0745, 0.0862, 0.0781],
       [0.934 , 0.9288, 0.8966, ..., 0.0679, 0.0812, 0.0715],
       ...,
       [0.1842, 0.2114, 0.1636, ..., 0.139 , 0.1236, 0.1261],
       [0.1353, 0.1076, 0.0985, ..., 0.1375, 0.1232, 0.1175],
       [0.069 , 0.134 , 0.237 , ..., 0.1372, 0.1248, 0.1179]])

In [22]:
green

<xarray.DataArray 'b03' (y: 10980, x: 10980)> Size: 964MB
dask.array<rechunk-merge, shape=(10980, 10980), dtype=float64, chunksize=(512, 512), chunktype=numpy.ndarray>
Coordinates:
  * x        (x) int64 88kB 300005 300015 300025 300035 ... 409775 409785 409795
  * y        (y) int64 88kB 5000035 5000025 5000015 ... 4890265 4890255 4890245
Attributes:
    _eopf_attrs:     {'add_offset': -0.1, 'coordinates': ['x', 'y'], 'dimensi...
    dtype:           <u2
    fill_value:      0
    long_name:       TOA reflectance from MSI acquisition at spectral band b0...
    proj:bbox:       [300000.0, 4890240.0, 409800.0, 5000040.0]
    proj:epsg:       32632
    proj:shape:      [10980, 10980]
    proj:transform:  [10.0, 0.0, 300000.0, 0.0, -10.0, 5000040.0, 0.0, 0.0, 1.0]
    proj:wkt2:       PROJCS["WGS 84 / UTM zone 32N",GEOGCS["WGS 84",DATUM["WG...
    units:           digital_counts
    valid_max:       65535
    valid_min:       1

In [32]:

def apply_rio_color(ops, channel, c_red, c_green, c_blue):
    arr = np.stack([np.clip(c_red, 0, 1), 
                    np.clip(c_green, 0, 1), 
                    np.clip(c_blue, 0, 1)], axis=0)
    assert arr.shape[0] == 3
    assert np.nanmin(arr) >= 0, "Input values must be >= 0"
    assert np.nanmax(arr) <= 1, "Input values must be <= 1"
    for func in parse_operations(ops):
        arr = func(arr)
    return (arr[channel, :, :] * 255).astype(np.uint8)

def apply_rio_color_red(ops, c_red, c_green, c_blue):
    return apply_rio_color(ops, 0, c_red, c_green, c_blue)

def apply_rio_color_green(ops, c_red, c_green, c_blue):
    return apply_rio_color(ops, 1, c_red, c_green, c_blue)

def apply_rio_color_blue(ops, c_red, c_green, c_blue):
    return apply_rio_color(ops, 2, c_red, c_green, c_blue)


In [24]:
ops = "gamma b 1.85, gamma rg 1.95" #, sigmoidal rgb 35 0.13"# , saturation 1.15"

# Apply the color correction using Dask and xarray
color_corrected_red = xr.apply_ufunc(
    apply_rio_color_red,
    ops, red, green, blue,
    dask='parallelized',
)

color_corrected_green = xr.apply_ufunc(
    apply_rio_color_green,
    ops, red, green, blue,
    dask='parallelized',
    output_dtypes=[np.uint8],
)

color_corrected_blue = xr.apply_ufunc(
    apply_rio_color_blue,
    red, green, blue,
    dask='parallelized',
    output_dtypes=[np.uint8],
)

# Combine the corrected channels into a single array
stacked = xr.concat([color_corrected_red, color_corrected_green, color_corrected_blue], dim='band')

# Set attributes for GeoTIFF
stacked = stacked.rio.write_crs(f"EPSG:{dt.attrs.get('stac_discovery').get('properties').get('proj:epsg')}", inplace=True)
stacked = stacked.rio.set_spatial_dims('x', 'y', inplace=True)

# Trigger the computation and save as GeoTIFF
stacked.compute().rio.to_raster('color_corrected.tif')

In [25]:
ndvi = ndvi.rio.write_crs(f"EPSG:{dt.attrs.get('stac_discovery').get('properties').get('proj:epsg')}", inplace=True)
ndvi.compute().rio.to_raster('ndvi.tif')

In [26]:
ndwi = ndwi.rio.write_crs(f"EPSG:{dt.attrs.get('stac_discovery').get('properties').get('proj:epsg')}", inplace=True)
ndwi.compute().rio.to_raster('ndwi.tif')

In [27]:
dt.attrs.get("stac_discovery").get("properties").get("proj:epsg")

32632

In [28]:
dt.attrs.get("stac_discovery").get("assets").get("analytic").get("eo:bands")

[{'center_wavelength': 0.4423,
  'common_name': 'coastal',
  'full_width_half_max': 0.02,
  'name': '01',
  'solar_illumination': '1874.3'},
 {'center_wavelength': 0.4923,
  'common_name': 'blue',
  'full_width_half_max': 0.065,
  'name': '02',
  'solar_illumination': '1959.75'},
 {'center_wavelength': 0.559,
  'common_name': 'green',
  'full_width_half_max': 0.035,
  'name': '03',
  'solar_illumination': '1824.93'},
 {'center_wavelength': 0.665,
  'common_name': 'red',
  'full_width_half_max': 0.03,
  'name': '04',
  'solar_illumination': '1512.79'},
 {'center_wavelength': 0.7038,
  'full_width_half_max': 0.015,
  'name': '05',
  'solar_illumination': '1425.78'},
 {'center_wavelength': 0.7391000000000001,
  'full_width_half_max': 0.015,
  'name': '06',
  'solar_illumination': '1291.13'},
 {'center_wavelength': 0.7797000000000001,
  'full_width_half_max': 0.02,
  'name': '07',
  'solar_illumination': '1175.57'},
 {'center_wavelength': 0.833,
  'common_name': 'nir',
  'full_width_half_m

In [34]:
# VEA
# Apply the color correction using Dask and xarray
ops = "gamma b 1.85, gamma rg 1.95"

color_corrected_red = xr.apply_ufunc(
    apply_rio_color_red,
    swir16_resampled, nir, red,
    dask='parallelized',
)

color_corrected_green = xr.apply_ufunc(
    apply_rio_color_green,
    swir16_resampled, nir, red,
    dask='parallelized',
    output_dtypes=[np.uint8],
)

color_corrected_blue = xr.apply_ufunc(
    apply_rio_color_blue,
    swir16_resampled, nir, red,
    dask='parallelized',
    output_dtypes=[np.uint8],
)

# Combine the corrected channels into a single array
stacked = xr.concat([color_corrected_red, color_corrected_green, color_corrected_blue], dim='band')

# Set attributes for GeoTIFF
stacked = stacked.rio.write_crs(f"EPSG:{dt.attrs.get('stac_discovery').get('properties').get('proj:epsg')}", inplace=True)
stacked = stacked.rio.set_spatial_dims('x', 'y', inplace=True)

# Trigger the computation and save as GeoTIFF
stacked.compute().rio.to_raster('vea.tif')

/tmp/ipykernel_2740130/673680715.py:12: RuntimeWarning: invalid value encountered in cast
/tmp/ipykernel_2740130/673680715.py:12: RuntimeWarning: invalid value encountered in cast
/tmp/ipykernel_2740130/673680715.py:12: RuntimeWarning: invalid value encountered in cast
/tmp/ipykernel_2740130/673680715.py:12: RuntimeWarning: invalid value encountered in cast
/tmp/ipykernel_2740130/673680715.py:12: RuntimeWarning: invalid value encountered in cast
/tmp/ipykernel_2740130/673680715.py:12: RuntimeWarning: invalid value encountered in cast
/tmp/ipykernel_2740130/673680715.py:12: RuntimeWarning: invalid value encountered in cast
/tmp/ipykernel_2740130/673680715.py:12: RuntimeWarning: invalid value encountered in cast
/tmp/ipykernel_2740130/673680715.py:12: RuntimeWarning: invalid value encountered in cast
/tmp/ipykernel_2740130/673680715.py:12: RuntimeWarning: invalid value encountered in cast
/tmp/ipykernel_2740130/673680715.py:12: RuntimeWarning: invalid value encountered in cast
/tmp/ipyke